# Heuristics Tests for the Traveling Tournament Problem (TTP)
This notebook tests several heuristics, meta-heuristics, and hybrid methods to solve (usually large) instances of the conventional TTP.

In [2]:
# Import libraries

import copy
import gurobipy as gp
# import json
import math
import pprint
import random as rd
import xmltodict

from gurobipy import GRB

In [3]:
# !pip install --upgrade gurobipy

## Parse Data

In this section, we parse XML files downloaded from the following site: https://robinxval.ugent.be/RobinX/travelRepo.php

Each XML has information pertaining a specific instance of the TTP. These can be converted to JSON format, and then to Python dictionaries, for easier management of the data within. 

In [4]:
# Convert XML data to JSON / Python dictionary

ttp = xmltodict.parse(open('ttp_instances/NL8.xml').read())
# ttp = json.dumps(ttp)
print(ttp)

{'Instance': {'MetaData': {'InstanceName': 'NL8', 'DataType': 'A', 'Contributor': 'Easton, Nemhauser, and Trick', 'Date': {'@day': '0', '@month': '0', '@year': '2001'}, 'Country': None, 'Remarks': 'Based on National Hockey League', 'Lowerbound': {'@infeasibility': '0', '@objective': '0'}}, 'Structure': {'Format': {'@leagueIds': '0', 'numberRoundRobin': '2', 'compactness': 'C'}, 'AdditionalGames': None}, 'ObjectiveFunction': {'Objective': 'TR'}, 'Data': {'Distances': {'distance': [{'@dist': '0', '@team1': '4', '@team2': '4'}, {'@dist': '1020', '@team1': '4', '@team2': '2'}, {'@dist': '1090', '@team1': '4', '@team2': '1'}, {'@dist': '957', '@team1': '4', '@team2': '6'}, {'@dist': '1380', '@team1': '4', '@team2': '3'}, {'@dist': '1190', '@team1': '4', '@team2': '7'}, {'@dist': '605', '@team1': '4', '@team2': '0'}, {'@dist': '1010', '@team1': '4', '@team2': '5'}, {'@dist': '1020', '@team1': '2', '@team2': '4'}, {'@dist': '0', '@team1': '2', '@team2': '2'}, {'@dist': '80', '@team1': '2', '@

In [5]:
# Obtain relevant data

# Distances
distances = dict()
for distance in ttp['Instance']['Data']['Distances']['distance']:
    distances[(int(distance['@team1']), int(distance['@team2']))] = int(distance['@dist'])
print(distances)

# Teams
teams = dict()
for team in ttp['Instance']['Resources']['Teams']['team']:
    teams[int(team['@id'])] = team['@name']
print(teams)

{(4, 4): 0, (4, 2): 1020, (4, 1): 1090, (4, 6): 957, (4, 3): 1380, (4, 7): 1190, (4, 0): 605, (4, 5): 1010, (2, 4): 1020, (2, 2): 0, (2, 1): 80, (2, 6): 501, (2, 3): 380, (2, 7): 664, (2, 0): 665, (2, 5): 257, (1, 4): 1090, (1, 2): 80, (1, 1): 0, (1, 6): 567, (1, 3): 337, (1, 7): 712, (1, 0): 745, (1, 5): 315, (6, 4): 957, (6, 2): 501, (6, 1): 567, (6, 6): 0, (6, 3): 622, (6, 7): 250, (6, 0): 370, (6, 5): 253, (3, 4): 1380, (3, 2): 380, (3, 1): 337, (3, 6): 622, (3, 3): 0, (3, 7): 646, (3, 0): 929, (3, 5): 408, (7, 4): 1190, (7, 2): 664, (7, 1): 712, (7, 6): 250, (7, 3): 646, (7, 7): 0, (7, 0): 587, (7, 5): 410, (0, 4): 605, (0, 2): 665, (0, 1): 745, (0, 6): 370, (0, 3): 929, (0, 7): 587, (0, 0): 0, (0, 5): 521, (5, 4): 1010, (5, 2): 257, (5, 1): 315, (5, 6): 253, (5, 3): 408, (5, 7): 410, (5, 0): 521, (5, 5): 0}
{0: 'ATL', 1: 'NYM', 2: 'PHI', 3: 'MON', 4: 'FLA', 5: 'PIT', 6: 'CIN', 7: 'CHI'}


## Greedy Algorithm
We use this algorithm to find an initial feasible solution to the TTP (assuming every team can directly travel to every other team's home).

In [6]:
# Greedy algorithm

def greedy_algorithm(n: int):
    """
    Greedy algorithm for TTP data.
    Input:
        - n: int --> number of participating teams (must be even for a feasible round-robin)
    Output:
        - initial_sol: dict --> initial solution for double round-robin, where keys are rounds and values are lists of matchups
            Notes:
                - each matchup is represented as a tuple of the form (home_team, away_team)
                - there are 2*(n-1) total rounds in a double round-robin, where n/2 matchups are played in each round
    """
    
    # Initialize solution dictionary with single round-robin matchups
    initial_sol = {i: [] for i in range(n - 1)}
    
    # Create list of lexopgraphical matchups    
    lexographical_matchups = [(i, j) for i in range(n - 1) for j in range(i + 1, n)]
    
    # Fill solution dictionary with matchups
    current_round = 0
    while lexographical_matchups:
        home_team, away_team = lexographical_matchups[0]
        
        # Get all teams currently scheduled in the target round
        teams_in_round = set()
        for existing_match in initial_sol[current_round]:
            teams_in_round.add(existing_match[0])
            teams_in_round.add(existing_match[1])
        
        # Check for repeated teams in round
        if home_team not in teams_in_round and away_team not in teams_in_round:
            initial_sol[current_round].append((home_team, away_team))   # Add matchup to round
            lexographical_matchups.pop(0)   # Remove matchup from lexographical list
        current_round = (current_round + 1) % (n - 1)   # Move to next round (or first round if at last round)
    
    # Add mirrored results to complete the double round-robin
    for round_num in range(n - 1):
        initial_sol[round_num + n - 1] = [matchup[::-1] for matchup in initial_sol[round_num]]
    
    return initial_sol 

In [7]:
# Tests

n = len(teams)
lexographical_matchups = [(i, j) for i in range(n - 1) for j in range(i + 1, n)]
print(lexographical_matchups)

initial_sol = greedy_algorithm(n)
print(initial_sol)

[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 4), (3, 5), (3, 6), (3, 7), (4, 5), (4, 6), (4, 7), (5, 6), (5, 7), (6, 7)]
{0: [(0, 1), (2, 6), (3, 5), (4, 7)], 1: [(0, 2), (1, 7), (3, 6), (4, 5)], 2: [(0, 3), (1, 2), (4, 6), (5, 7)], 3: [(0, 4), (1, 3), (2, 7), (5, 6)], 4: [(0, 5), (1, 4), (2, 3), (6, 7)], 5: [(0, 6), (1, 5), (2, 4), (3, 7)], 6: [(0, 7), (1, 6), (2, 5), (3, 4)], 7: [(1, 0), (6, 2), (5, 3), (7, 4)], 8: [(2, 0), (7, 1), (6, 3), (5, 4)], 9: [(3, 0), (2, 1), (6, 4), (7, 5)], 10: [(4, 0), (3, 1), (7, 2), (6, 5)], 11: [(5, 0), (4, 1), (3, 2), (7, 6)], 12: [(6, 0), (5, 1), (4, 2), (7, 3)], 13: [(7, 0), (6, 1), (5, 2), (4, 3)]}


## Simulated Annealing / Tabu Search

We have a solution space given by coloring schemes of a graph G, such that:
- each vertex v in G represents a possible matchup (for n teams in a double round robin, there are n*(n-1) possible matchups – excluding impossible matchups of a team competing against itself) 
- each edge e in G is placed between vertices that share at least one team in their matchups
- we use 2*(n - 1) colors for the vertices of G, such that neighboring vertices cannot share the same color
    - here, each color represents the matchups within each of the 2*(n-1) rounds in a double round-robin

To travel along this solution space, we use the neighboring moves described in the paper "An Efficient Simulated Annealing Approach to the Travelling Tournament Problem". Each move is chosen at random* based on the possible choices, and may represent an improving or non-improving difference of a solution's value compared to its predecesor. We determine whether to make the move or not with the simulated annealing meta-heuristic, which proposes a temperature T such that:
- if the value of the new solution is improving on our optimization function --> we make the move
- if the value of the new solution is non-improving on our optimization function --> we have a probability of exp(-delta/T), where delta is the difference in values of our previous and new solutions
- the temperature parameter T decreases over the number of iterations (implying that the probability of choosing non-improving moves decreases over time). The rate of decrease is given by a fixed 'cooling parameter'. 

*more info on this in the paper

### Auxiliary Functions

In [8]:
# Functions for neighboring moves

def swap_rounds(sol: dict, round_1: int, round_2: int):
    """
    Swap two rounds in a solution.
    Input:
        - sol: dict --> solution for double round-robin, where keys are rounds and values are lists of matchups
        - round1, round2: int --> rounds to be swapped
    Output:
        - new_sol: dict --> new solution where round1 is changed to round2, and viceversa
    """
    
    if round_1 == round_2:
        return sol
    
    new_sol = copy.deepcopy(sol)
    new_sol[round_1] = sol[round_2]
    new_sol[round_2] = sol[round_1]
    
    return new_sol


def swap_homes(sol: dict, team_1: int, team_2: int):
    """
    Swap the homes of two teams in a solution.
    Input:
        - sol: dict --> solution for double round-robin, where keys are rounds and values are lists of matchups
        - team_1, team_2: int --> teams to be swapped
    Output:
        - new_sol: dict --> new solution where (team_1, team_2) is changed to (team_2, team_1), and viceversa
    """
    
    if team_1 == team_2:
        return sol
    
    new_sol = copy.deepcopy(sol)
    counter = 0
    for round in sol:
        if (team_1, team_2) in sol[round]:
            new_sol[round].remove((team_1, team_2))
            new_sol[round].append((team_2, team_1))
            counter += 1
        elif (team_2, team_1) in sol[round]:
            new_sol[round].remove((team_2, team_1))
            new_sol[round].append((team_1, team_2))
            counter += 1
        if counter == 2:
            break
    
    return new_sol


def swap_teams(sol: dict, team_1: int, team_2: int):
    """
    Swap the positions of two teams in a solution, except for the games they play against each other.
    Input:
        - sol: dict --> solution for double round-robin, where keys are rounds and values are lists of matchups
        - team_1, team_2: int --> teams with schedules to be swapped
    Output:
        - new_sol: dict --> new solution where the positions of team_1 and team_2 are swapped in each round, except when they play against each other
    """
    
    if team_1 == team_2:
        return sol
    
    new_sol = copy.deepcopy(sol)
    for round in sol:
        if ((team_1, team_2) not in sol[round]) and ((team_2, team_1) not in sol[round]):
            updated_matches = []
            for team_x, team_y in sol[round]:
                if team_x == team_1:
                    updated_matches.append((team_2, team_y))
                elif team_x == team_2:
                    updated_matches.append((team_1, team_y))
                elif team_y == team_1:
                    updated_matches.append((team_x, team_2))
                elif team_y == team_2:
                    updated_matches.append((team_x, team_1))
                else:
                    updated_matches.append((team_x, team_y))
            new_sol[round] = updated_matches
    
    return new_sol


def swap_rounds_partial(sol: dict, team: int, round_1: int, round_2: int):
    """
    Swap the matches containing a specific team in two rounds of a solution, then update these rounds to mantain a valid solution.
    Input:
        - sol: dict --> solution for double round-robin, where keys are rounds and values are lists of matchups
        - team: int --> team's matches to be swapped
        - round1, round2: int --> rounds in which the matches of team are to be swapped
    Output:
        - new_sol: dict --> new solution where matches containing the given team are swapped between round1 and round2
    """
    
    if round_1 == round_2:
        return sol
    
    new_sol = copy.deepcopy(sol)
    matches_to_swap = []
    swapped_teams = {"to_r1": [], "to_r2": []}
    
    # Find the matches (containing 'team') to be swapped between round_1 and round_2, and find the competing teams in these matches
    for round in [round_1, round_2]:
        for team_x, team_y in sol[round]:
            if (team_x == team) or (team_y == team):
                matches_to_swap.append((team_x, team_y))
                if round == round_1:
                    swapped_teams["to_r2"].append(team_x)
                    swapped_teams["to_r2"].append(team_y)
                else:
                    swapped_teams["to_r1"].append(team_x)
                    swapped_teams["to_r1"].append(team_y)
                break
    
    # Update rounds for a valid solution by swapping more matches between round_1 and round_2
    while not set(swapped_teams["to_r1"]) == set(swapped_teams["to_r2"]):
        for round in [round_1, round_2]:
            for team_x, team_y in sol[round]:
                if round == round_1:
                    if ((team_x, team_y) not in matches_to_swap) and ((team_x in swapped_teams["to_r1"]) or (team_y in swapped_teams["to_r1"])):
                        matches_to_swap.append((team_x, team_y))
                        swapped_teams["to_r2"].append(team_x)
                        swapped_teams["to_r2"].append(team_y)
                else:
                    if ((team_x, team_y) not in matches_to_swap) and ((team_x in swapped_teams["to_r2"]) or (team_y in swapped_teams["to_r2"])):
                        matches_to_swap.append((team_x, team_y))
                        swapped_teams["to_r1"].append(team_x)
                        swapped_teams["to_r1"].append(team_y)
    
    # Update new_sol with swapped matches
    for round in [round_1, round_2]:
        for team_x, team_y in sol[round]:
            if (team_x, team_y) in matches_to_swap:
                new_sol[round].remove((team_x, team_y))
                if round == round_1:
                    new_sol[round_2].append((team_x, team_y))
                else:
                    new_sol[round_1].append((team_x, team_y))
    
    return new_sol


def swap_teams_partial(sol: dict, round: int, team_1: int, team_2: int):
    """
    Swap the positions of two teams in a given round, then update affected rounds to mantain a valid solution.
    Input:
        - sol: dict --> solution for double round-robin, where keys are rounds and values are lists of matchups
        - round: int --> round in which the opponents of team_1 and team_2 are to be swapped
        - team_1, team_2: int --> teams to be swapped
    Output:
        - new_sol: dict --> new solution where the positions of team_1 and team_2 are swapped in the given round
    """
    
    if team_1 == team_2:
        return sol
    
    def swap_teams_2(sol: dict, team_1: int, team_2: int):
        """Same as swap_teams, but also returns the new matches of team_1 and team_2 after the swaps, which is needed for swap_teams_partial."""
        new_sol = copy.deepcopy(sol)
        new_matches = []
        for round in sol:
            if ((team_1, team_2) not in sol[round]) and ((team_2, team_1) not in sol[round]):
                updated_matches = []
                for team_x, team_y in sol[round]:
                    if team_x == team_1:
                        updated_matches.append((team_2, team_y))
                        new_matches.append((team_2, team_y))
                    elif team_x == team_2:
                        updated_matches.append((team_1, team_y))
                        new_matches.append((team_1, team_y))
                    elif team_y == team_1:
                        updated_matches.append((team_x, team_2))
                        new_matches.append((team_x, team_2))
                    elif team_y == team_2:
                        updated_matches.append((team_x, team_1))
                        new_matches.append((team_x, team_1))
                    else:
                        updated_matches.append((team_x, team_y))
                new_sol[round] = updated_matches
        
        return new_sol, new_matches
    
    # Set initial variables
    new_sol = copy.deepcopy(sol)
    rounds_not_visited = list(sol.keys())
    affected_rounds = [round]
    
    # Swap positions of team_1 and team_2 in affected rounds, and remove the affected rounds from rounds_not_visited, while possible
    while affected_rounds != []:
        swapped_teams, new_matches = swap_teams_2({r : sol[r] for r in affected_rounds}, team_1, team_2)
        #print("Swapped teams:", swapped_teams)
        if new_matches == []:   # case where team_1 and team_2 play against each other
            return new_sol
        for round in affected_rounds:
            new_sol[round] = swapped_teams[round]
            #print("Updated solution:", new_sol)
            rounds_not_visited.remove(round)
        
        # Identify new affected rounds
        affected_rounds = []
        for match in new_matches:
            for round in {r : new_sol[r] for r in rounds_not_visited}:
                if match in new_sol[round]:
                    if round not in affected_rounds:
                            affected_rounds.append(round)
                    break
    
    return new_sol

In [9]:
# Tests

n = len(teams)
initial_sol = greedy_algorithm(n)
print("Initial solution:", initial_sol, "\n")

new_sol = swap_homes(initial_sol, 0, 1)
print("Swap homes:", new_sol)

new_sol = swap_rounds(initial_sol, 0, 1)
print("Swap rounds:", new_sol)

new_sol = swap_teams(initial_sol, 0, 1)
print("Swap teams:", new_sol)

new_sol = swap_rounds_partial(initial_sol, 0, 0, 1)
print("Swap rounds partial:", new_sol)

new_sol = swap_teams_partial(initial_sol, 0, 0, 2)
print("Swap teams partial:", new_sol)

Initial solution: {0: [(0, 1), (2, 6), (3, 5), (4, 7)], 1: [(0, 2), (1, 7), (3, 6), (4, 5)], 2: [(0, 3), (1, 2), (4, 6), (5, 7)], 3: [(0, 4), (1, 3), (2, 7), (5, 6)], 4: [(0, 5), (1, 4), (2, 3), (6, 7)], 5: [(0, 6), (1, 5), (2, 4), (3, 7)], 6: [(0, 7), (1, 6), (2, 5), (3, 4)], 7: [(1, 0), (6, 2), (5, 3), (7, 4)], 8: [(2, 0), (7, 1), (6, 3), (5, 4)], 9: [(3, 0), (2, 1), (6, 4), (7, 5)], 10: [(4, 0), (3, 1), (7, 2), (6, 5)], 11: [(5, 0), (4, 1), (3, 2), (7, 6)], 12: [(6, 0), (5, 1), (4, 2), (7, 3)], 13: [(7, 0), (6, 1), (5, 2), (4, 3)]} 

Swap homes: {0: [(2, 6), (3, 5), (4, 7), (1, 0)], 1: [(0, 2), (1, 7), (3, 6), (4, 5)], 2: [(0, 3), (1, 2), (4, 6), (5, 7)], 3: [(0, 4), (1, 3), (2, 7), (5, 6)], 4: [(0, 5), (1, 4), (2, 3), (6, 7)], 5: [(0, 6), (1, 5), (2, 4), (3, 7)], 6: [(0, 7), (1, 6), (2, 5), (3, 4)], 7: [(6, 2), (5, 3), (7, 4), (0, 1)], 8: [(2, 0), (7, 1), (6, 3), (5, 4)], 9: [(3, 0), (2, 1), (6, 4), (7, 5)], 10: [(4, 0), (3, 1), (7, 2), (6, 5)], 11: [(5, 0), (4, 1), (3, 2), (7, 6)]

In [10]:
# Auxiliary functions

def calculate_total_distance(sol: dict, teams: dict, distances: dict):
    """
    Calculate total distance for feasible solutions.
    Input:
        - sol: dict --> solution for double round-robin, where keys are rounds and values are lists of matchups
        - teams: dict --> dictionary of team homes
        - distances: dict --> dictionary of distances between team homes
    Output:
        - total_distance: float --> total distance travelled by all teams in the solution
    
    Note: this function assumes that the shortest path between two teams is the direct path between their homes
    """
    
    total_distance = 0
    for team in teams.keys():
        current_location = team
        for round in sol:
            for team_x, team_y in sol[round]:
                if team_y == team:
                    if current_location != team:
                        total_distance += distances[(current_location, team_x)]
                        current_location = team_x
                    else:
                        total_distance += distances[(team_y, team_x)]
                        current_location = team_x
                if (team_x == team) and (current_location != team):
                    total_distance += distances[(current_location, team_x)]
                    current_location = team_x
            if round == len(sol) - 1:
                total_distance += distances[(current_location, team)]   
    
    return total_distance


def violated_constraints(sol: dict, teams: dict, u: int = 3):
    """
    Checks how many times constraints of the TTP are violated in a solution (at-most, no-repeat).
    Input:
        - sol: dict --> solution for double round-robin, where keys are rounds and values are lists of matchups
        - teams: dict --> dictionary of team homes
        - u: int --> upper bound for the consecutive 'at home' or 'away' matches that a team is allowed to play
    Output:
        - num_violated_constraints: int --> number of times constraints are violated
    """
    
    consecutive_at_home = {i: 0 for i in range(len(teams))}
    consecutive_away = {i: 0 for i in range(len(teams))}
    num_violated_constraints = 0
    
    for round in range(len(sol) - 1):
        previous_matches = sol[round]
        for team_x, team_y in sol[round + 1]:
            if (team_y, team_x) in previous_matches:
                num_violated_constraints += 1
            if team_x in [same_team[0] for same_team in previous_matches]:
                consecutive_at_home[team_x] += 1
            if team_y in [same_team[0] for same_team in previous_matches]:
                consecutive_at_home[team_y] = 0
            if team_x in [same_team[1] for same_team in previous_matches]:
                consecutive_away[team_x] = 0
            if team_y in [same_team[1] for same_team in previous_matches]:
                consecutive_away[team_y] += 1
            
            if consecutive_at_home[team_x] >= u:
                num_violated_constraints += 1
            if consecutive_away[team_y] >= u:
                num_violated_constraints += 1
    
    return num_violated_constraints


def objective_function(sol: dict, teams: dict, distances: dict, w: float, func, u: int = 3):
    """
    Objective function for TTP data.
    Input:
        - sol: dict --> solution for double round-robin, where keys are rounds and values are lists of matchups
        - teams: dict --> dictionary of team homes
        - distances: dict --> dictionary of distances between team homes
        - w: float --> weight to control the impact of violated constraints
        - func: function --> function to control the impact of violated constraints
        - u: int --> upper bound for the consecutive 'at home' or 'away' matches that a team is allowed to play
    Output:
        - objective_value: float --> objective function's value
    """
    
    total_distance = calculate_total_distance(sol, teams, distances)
    num_violated_constraints = violated_constraints(sol, teams, u)
    
    if num_violated_constraints == 0:
        objective_value = total_distance
    else:
        objective_value = (total_distance**2 + (w * func(num_violated_constraints))**2)**(1/2)
    
    return objective_value


def choose_moves(sol, n_teams, n_moves = 1):
    """
    Randomly chooses one of the five possible moves (swap_rounds, swap_homes, swap_teams, swap_rounds_partial, swap_teams_partial) to be applied to the solution.
    Input:
        - sol: dict --> solution for double round-robin, where keys are rounds and values are lists of matchups
        - n_teams: int --> number of teams
        - n_moves: int --> number of moves to be applied (1 by default)
    Output:
        - sol: dict --> new solution for double round-robin after applying a move
    """
    
    for i in range(n_moves):
        team_1 = rd.randint(0, n_teams - 1)
        team_2 = rd.randint(0, n_teams - 1)
        round_1 = rd.randint(0, len(sol) - 1)
        round_2 = rd.randint(0, len(sol) - 1)
        choice = rd.randint(0, 4)
        if choice == 0:
            sol = swap_rounds(sol, round_1, round_2)
        elif choice == 1:
            sol = swap_homes(sol, team_1, team_2)
        elif choice == 2:
            sol = swap_teams(sol, team_1, team_2)
        elif choice == 3:
            sol = swap_rounds_partial(sol, team_1, round_1, round_2)
        elif choice == 4:
            sol = swap_teams_partial(sol, round_1, team_1, team_2)
    
    return sol

In [11]:
# Tests

def constraint_penalty(num_violated_constraints: int):
    x = num_violated_constraints
    return 1 + (x**(1/2) * math.log(x)) / 2

n = len(teams)
initial_sol = greedy_algorithm(n)
print("Initial solution:", initial_sol)

randomized_sol = choose_moves(initial_sol, n, 100)
print("Randomized solution:", randomized_sol)

total_distance = calculate_total_distance(randomized_sol, teams, distances)
print("Total distance:", total_distance)

num_violated_constraints = violated_constraints(randomized_sol, teams)
print("Violated constraints:", num_violated_constraints)

objective_value = objective_function(randomized_sol, teams, distances, 3000, constraint_penalty)
print("Objective value:", objective_value)

Initial solution: {0: [(0, 1), (2, 6), (3, 5), (4, 7)], 1: [(0, 2), (1, 7), (3, 6), (4, 5)], 2: [(0, 3), (1, 2), (4, 6), (5, 7)], 3: [(0, 4), (1, 3), (2, 7), (5, 6)], 4: [(0, 5), (1, 4), (2, 3), (6, 7)], 5: [(0, 6), (1, 5), (2, 4), (3, 7)], 6: [(0, 7), (1, 6), (2, 5), (3, 4)], 7: [(1, 0), (6, 2), (5, 3), (7, 4)], 8: [(2, 0), (7, 1), (6, 3), (5, 4)], 9: [(3, 0), (2, 1), (6, 4), (7, 5)], 10: [(4, 0), (3, 1), (7, 2), (6, 5)], 11: [(5, 0), (4, 1), (3, 2), (7, 6)], 12: [(6, 0), (5, 1), (4, 2), (7, 3)], 13: [(7, 0), (6, 1), (5, 2), (4, 3)]}
Randomized solution: {0: [(5, 0), (6, 3), (4, 2), (7, 1)], 1: [(3, 1), (4, 0), (2, 7), (6, 5)], 2: [(4, 1), (2, 6), (0, 5), (7, 3)], 3: [(1, 4), (6, 0), (2, 5), (3, 7)], 4: [(4, 7), (6, 2), (1, 0), (5, 3)], 5: [(7, 0), (4, 6), (2, 1), (3, 5)], 6: [(3, 0), (7, 6), (5, 4), (1, 2)], 7: [(7, 2), (4, 3), (0, 6), (5, 1)], 8: [(0, 1), (2, 3), (5, 7), (6, 4)], 9: [(5, 2), (3, 6), (0, 4), (1, 7)], 10: [(5, 6), (7, 4), (1, 3), (2, 0)], 11: [(0, 7), (6, 1), (4, 5), 

### Simulated Annealing heuristic

In [12]:
# Simulated annealing heuristic

def simulated_annealing(initial_sol: dict, params: dict, temperature: float, max_r: int, max_p: int, max_c: int,
                        cooling_rate: float = 0.999, reheating_rate: float = 2, w_modifier: float = 1.04, initial_sol_type: str = 'greedy'):
    """
    Simulated annealing algorithm for TTP data.
    Input:
        - initial_sol: dict --> initial solution for double round-robin, where keys are rounds and values are lists of matchups
        - params: dict --> parameters to be given to the objective function (except the solution)
        - temperature: float --> initial temperature
        - max_r: int --> maximum number of reheats
        - max_p: int --> maximum number of phases (iterations per reheat)
        - max_c: int --> upper bound for the counter of non-improving moves
        - cooling_rate: float --> cooling rate (0.999 by default)
        - reheating_rate: float --> reheating rate (2 by default)
        - w_modifier: float --> weight modifier (0.1 by default)
    Output:
        - final_sol: dict --> final solution for double round-robin (ideally, optimal or close to optimal for the TTP)
        - total_distance: float --> total distance travelled by all teams in the final solution
    """
    
    n_teams = len(params["teams"])
    
    # Randomize initial solution
    if initial_sol_type == 'greedy':
        sol = choose_moves(initial_sol, n_teams, 100)
    else:
        sol = initial_sol
    
    # Set initial parameters
    best_feasible = math.inf
    new_best_feasible = math.inf
    best_infeasible = math.inf
    new_best_infeasible = math.inf
    best_sol = initial_sol
    reheat = 0

    # Run simulated annealing
    while reheat < max_r:
        phase = 0
        while phase < max_p:
            non_improving_moves = 0
            while non_improving_moves < max_c:
                new_sol = choose_moves(sol, n_teams, 1)
                if (objective_function(new_sol, **params) < objective_function(sol, **params)) or ((violated_constraints(new_sol, params['teams'], params['u']) == 0) and (objective_function(new_sol, **params) < best_feasible)) or ((violated_constraints(new_sol, params['teams'], params['u']) > 0) and (objective_function(new_sol, **params) < best_infeasible)):
                    accept = True
                else:
                    if rd.random() < math.exp((objective_function(sol, **params) - objective_function(new_sol, **params)) / temperature):
                        accept = True
                    else:
                        accept = False
                if accept:
                    sol = new_sol
                    if violated_constraints(sol, params['teams'], params['u']) == 0:
                        new_best_feasible = min(objective_function(sol, **params), best_feasible)
                    else:
                        new_best_infeasible = min(objective_function(sol, **params), best_infeasible)
                    if (new_best_feasible < best_feasible) or (new_best_infeasible < best_infeasible):
                        reheat = 0
                        phase = 0
                        non_improving_moves = 0
                        best_temperature = temperature
                        best_feasible = new_best_feasible
                        best_infeasible = new_best_infeasible
                        if violated_constraints(sol, params['teams'], params['u']) == 0:
                            params['w'] = params['w'] / w_modifier
                            best_sol = sol
                        else:
                            params['w'] = params['w'] * w_modifier
                    else:
                        non_improving_moves += 1
            phase += 1
            temperature *= cooling_rate
        reheat += 1
        temperature = reheating_rate * best_temperature
                        
    # Return final solution
    return best_sol

In [13]:
# Tests

def constraint_penalty(num_violated_constraints: int):
    x = num_violated_constraints
    return 1 + (x**(1/2) * math.log(x)) / 2

n = len(teams)
#initial_sol = greedy_algorithm(n)
initial_sol = {0: [(0, 1), (4, 2), (6, 3), (7, 5)], 1: [(0, 3), (1, 2), (4, 5), (6, 7)], 2: [(0, 6), (3, 1), (4, 7), (5, 2)], 3: [(2, 1), (5, 3), (6, 4), (7, 0)], 4: [(2, 3), (5, 1), (6, 0), (7, 4)], 5: [(1, 3), (2, 5), (4, 0), (7, 6)], 6: [(0, 5), (1, 7), (3, 2), (4, 6)], 7: [(0, 7), (1, 5), (2, 6), (3, 4)], 8: [(2, 7), (3, 0), (5, 4), (6, 1)], 9: [(4, 1), (5, 0), (6, 2), (7, 3)], 10: [(0, 2), (4, 3), (6, 5), (7, 1)], 11: [(0, 4), (1, 6), (3, 5), (7, 2)], 12: [(1, 0), (2, 4), (3, 6), (5, 7)], 13: [(1, 4), (2, 0), (3, 7), (5, 6)]}
params = {
    "distances": distances,
    "teams": teams,
    "w": n * 1000,
    "func": constraint_penalty,
    "u": 3
}

temperature = 400
max_r = 1
max_p = 50
max_c = 50

final_sol = simulated_annealing(initial_sol, params, temperature, max_r, max_p, max_c, cooling_rate=0.98, initial_sol_type='')
print("Final solution:", final_sol)
# 41,841
total_distance = calculate_total_distance(final_sol, teams, distances)
print("Total distance:", total_distance)

Final solution: {0: [(1, 4), (3, 2), (7, 0), (5, 6)], 1: [(5, 0), (1, 6), (3, 4), (7, 2)], 2: [(1, 3), (7, 5), (4, 0), (6, 2)], 3: [(6, 7), (2, 3), (0, 5), (4, 1)], 4: [(0, 1), (4, 5), (2, 6), (3, 7)], 5: [(1, 2), (5, 7), (3, 6), (0, 4)], 6: [(3, 1), (6, 0), (2, 5), (7, 4)], 7: [(0, 2), (6, 4), (7, 3), (5, 1)], 8: [(4, 2), (0, 6), (5, 3), (1, 7)], 9: [(3, 0), (4, 6), (2, 7), (1, 5)], 10: [(1, 0), (2, 4), (3, 5), (7, 6)], 11: [(5, 4), (2, 0), (6, 3), (7, 1)], 12: [(5, 2), (0, 3), (4, 7), (6, 1)], 13: [(2, 1), (4, 3), (0, 7), (6, 5)]}
Total distance: 43151


### Integer Programming Formulation

We formulate 2 IP subproblems to improve solutions obtained from simulated annealing.

In [21]:
# Functions to parse previous data

def format_distances(distances: dict):
    new_distances = dict()
    for match in distances.keys():
        new_distances[(match[0] + 1, match[1] + 1)] = distances[match]
    
    return new_distances


def generate_input_x_schedule(sol: dict, n: int):
    input_x_schedule = dict()
    for round in sol:
        for team_x, team_y in sol[round]:
            input_x_schedule[(team_y + 1, team_x + 1, round + 1)] = 1
    
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            for k in range(1, 2*(n - 1) + 1):
                if (i, j, k) not in input_x_schedule.keys():
                    input_x_schedule[(i, j, k)] = 0
    
    return input_x_schedule


def get_sol_from_x_var(x_var: dict):
    new_sol = dict()
    for item in x_var.keys():
        team_x, team_y, round = item[2:-1].split(',')
        team_x = int(team_x) - 1
        team_y = int(team_y) - 1
        round = int(round) - 1
        if round not in new_sol:
            new_sol[round] = []
        new_sol[round].append((team_y, team_x))
    
    new_sol = dict(sorted(new_sol.items()))
    
    return new_sol

In [24]:
# Functions

def build_base_model(distances, N, L, U, DAYS, Teams, Days_X, Days_Y, Days_U, Days_L):
    """
    Constructs the base TTP subproblem model including variables, objective,
    and the core TTP constraints (1) through (9).
    """
    model = gp.Model("TTP_Subproblem")
    model.setParam('OutputFlag', 0) # makes the output cleaner
    model.setParam('TimeLimit', 300.0)
    # --- 3. Decision Variables ---
    # x[i, j, k]: 1 if team i hosts team j on day k (Binary)
    x = model.addVars(Teams, Teams, Days_X, vtype=GRB.BINARY, name="x")

    # z[i, j, k]: Auxiliary variable (Continuous)
    z = model.addVars(Teams, Teams, Days_X, vtype=GRB.BINARY, name="z")

    # y[t, i, j, k]: Travel variable for team t from location i to j on day k (Continuous, LB=0)
    # k is the index of the FIRST day in the transition (k to k+1)
    y = model.addVars(Teams, Teams, Teams, Days_Y, vtype=GRB.BINARY, lb=0, name="y")

    # --- 4. Objective Function ---
    # min sum(i,j) d_ij * x_ij1 + sum(t,i,j) sum(k=1 to 2n-3) d_ij * y_tijk + sum(i,j) d_ij * x_ij,2n-2
    obj = gp.quicksum(distances[i, j] * x[i, j, 1] for i in Teams for j in Teams) + \
          gp.quicksum(distances[i, j] * y[t, i, j, k]
                      for t in Teams for i in Teams for j in Teams for k in Days_Y) + \
          gp.quicksum(distances[j, i] * x[i, j, DAYS] for i in Teams for j in Teams)

    model.setObjective(obj, GRB.MINIMIZE)

    # --- 5. Constraints (TTP Core) ---

    # (1) No self-play
    model.addConstrs((x[i, i, k] == 0 for i in Teams for k in Days_X), name="No_Self_Play")

    # (2) Play one game per day 
    model.addConstrs((gp.quicksum(x[i, j, k] + x[j, i, k] for j in Teams) == 1
                      for i in Teams for k in Days_X), name="Play_One_Game_Per_Day")

    # (3) Play each team twice 
    model.addConstrs((gp.quicksum(x[i, j, k] for k in Days_X) == 1
                      for i in Teams for j in Teams if i != j), name="Play_Each_Team_Twice")

    # (4) Max home stay
    model.addConstrs((gp.quicksum(x[i, j, k + u] for j in Teams for u in range(U + 1)) <= U
                      for i in Teams for k in Days_U), name="Max_Home_Stay")
    model.addConstrs((L <= gp.quicksum(x[i, j, k + u] for j in Teams for u in range(U + 1))
                      for i in Teams for k in Days_U), name="Max_Home_Stay")

    # (5) Max road trip
    model.addConstrs((gp.quicksum(x[i, j, k + u] for i in Teams for u in range(U + 1)) <= U
                      for j in Teams for k in Days_U), name="Max_Road_Trip")
    model.addConstrs((L <= gp.quicksum(x[i, j, k + u] for i in Teams for u in range(U + 1))
                      for j in Teams for k in Days_U), name="Max_Road_Trip")
    
    # (6) No game repeats 2 times back to back
    model.addConstrs((x[i, j, k] + x[j, i, k] + x[i, j, k + 1] + x[j, i, k + 1] <= 1
                      for i in Teams for j in Teams for k in Days_Y if i != j),
                      name="No_Back_to_Back_Game")

    # --- 6. Driving behavior of the teams constraints ---

    # (7)
    model.addConstrs((z[i, i, k] == gp.quicksum(x[j, i, k] for j in Teams)
                       for i in Teams for k in Days_X), name="Def_z_home")

    # (8))
    model.addConstrs((z[i, j, k] == x[i, j, k]
                      for i in Teams for j in Teams for k in Days_X if i != j), name="Def_z_game")

    # (9)
    model.addConstrs((y[t, i, j, k] >= z[t, i, k] + z[t, j, k + 1] - 1
                      for t in Teams for i in Teams for j in Teams for k in Days_Y),
                     name="Def_y_travel_link")
    # (10)  Min home stay (Lower Bound L): Forces team i to be home at least L times over L+1 days.
    #model.addConstrs((gp.quicksum(x[i, j, k + l] for j in Teams for l in range(L + 1)) >= L
    #                  for i in Teams for k in Days_L), name="Min_Home_Stay_L")

    # (11) Min road trip (Lower Bound L): Forces team j to be away at least L times over L+1 days.
    #model.addConstrs((gp.quicksum(x[i, j, k + l] for i in Teams for l in range(L + 1)) >= L
    #                  for j in Teams for k in Days_L), name="Min_Road_Trip_L")

    return model, x, z, y


def solve_ttp_subproblem(mode: str, input_schedule: dict, base_model_params: dict):
    """
    Solves the TTP subproblem IP given an input schedule from local search,
    specialized by the optimization mode ('HA-Opt' or 'Non-HA-Opt').

    Args:
        mode (str): 'HA-Opt' or 'Non-HA-Opt'.
        distances (dict): The distance matrix D[i, j].
        input_schedule (dict): The initial solution x_tilde[(i, j, k)].
    """
    if mode not in ['HA-Opt', 'Non-HA-Opt']:
        print("Error: Mode must be 'HA-Opt' or 'Non-HA-Opt'.")
        return None
    
    distances = base_model_params["distances"]
    n = base_model_params["n"]
    l = base_model_params["l"]
    u = base_model_params["u"]
    days = base_model_params["days"]
    Teams = base_model_params["teams"]
    Days_X = base_model_params["days_x"]
    Days_Y = base_model_params["days_y"]
    Days_U = base_model_params["days_u"]
    Days_L = base_model_params["days_l"]
    
    # 1. Build the base model (variables, objective, and core constraints 1-9)
    model, x, z, y = build_base_model(distances, n, l, u, days, Teams, Days_X, Days_Y, Days_U, Days_L)

    # 2. get the input schedule
    for i in Teams:
        for j in Teams:
            for k in Days_X:
                x[i, j, k].Start = input_schedule[(i, j, k)]

    # 3. Add Subproblem-Specific Constraints (Constraints 12, 13, or 14 from the paper)
    if mode == 'HA-Opt':
        print("HA-Optimization Mode: Fixing Matchups, Optimizing Venues (Constraint 12)")
        for i in Teams:
            for j in Teams:
                if i < j:
                    for k in Days_X:
                        matchup_fixed_val = input_schedule[(i, j, k)] + input_schedule[(j, i, k)]
                        model.addConstr(x[i, j, k] + x[j, i, k] == matchup_fixed_val,
                                        name=f"FixMatchup_{i}_{j}_{k}")

    elif mode == 'Non-HA-Opt':
        print("Non-HA-Optimization Mode: Fixing Venues, Optimizing Matchups (Constraints 13 & 14)")
        # constraint 14
        for i in Teams:
            for k in Days_X:
                home_status_fixed = gp.quicksum(input_schedule[(i, j, k)] for j in Teams)
                model.addConstr(gp.quicksum(x[i, j, k] for j in Teams) == home_status_fixed,
                                name=f"FixHomeStatus_{i}_{k}")
        # constraint 13
        for j in Teams:
            for k in Days_X:
                # Calculate the required away status (0 or 1) from the input schedule
                away_status_fixed = gp.quicksum(input_schedule[(i, j, k)] for i in Teams)
                model.addConstr(gp.quicksum(x[i, j, k] for i in Teams) == away_status_fixed,
                                name=f"FixAwayStatus_{j}_{k}")

    # 4. Solve the model
    model.optimize()

    # 5. Output Results
    if model.status == GRB.OPTIMAL or model.status == GRB.TIME_LIMIT:
        print(f"Optimization Status: {model.Status}")
        print(f"Objective Value: {model.objVal:,.0f}")
        
        # Return the new schedule x, z, y values
        return {
            'x': {v.varName: v.X for v in x.values() if v.X > 0.5},
            'z': {v.varName: v.X for v in z.values() if v.X > 0.5},
            'y': {v.varName: v.X for v in y.values() if v.X > 0.001},
            'objVal': model.objVal
        }
    else:
        print(f"Model could not be solved to optimality/time limit. Status: {model.Status}")
        return None

In [25]:
# Tests

# Get test inputs
n = len(teams)
u = 3
l = 0

#initial_sol = greedy_algorithm(n)
initial_sol = final_sol.copy()
print("Initial solution:", initial_sol)

distance_initial_sol = calculate_total_distance(initial_sol, teams, distances)
print("Distance of initial solution:", distance_initial_sol)

formatted_distances = format_distances(distances)
print("Formatted distances:", formatted_distances)

input_x_schedule = generate_input_x_schedule(initial_sol, n)
print("Input x schedule:", input_x_schedule, "\n")

# Test 1: HA-Opt
ha_result = solve_ttp_subproblem(
    mode = 'HA-Opt',
    input_schedule = input_x_schedule,
    base_model_params = {
        'distances': formatted_distances,
        'n': n,
        'l': l,
        'u': u,
        'days': 2*(n - 1),
        'teams': range(1, n + 1),
        'days_x': range(1, 2*(n - 1) + 1),
        'days_y': range(1, 2*(n - 1)),
        'days_u': range(1, 2*(n - 1) - u + 1),
        'days_l': range(1, 2*(n - 1) - l + 1)
    }
)
print("HA-Opt Solution:", get_sol_from_x_var(ha_result["x"]))
print(ha_result["x"])
print(ha_result["objVal"])
print("Total distance of HA-Opt solution:", calculate_total_distance(get_sol_from_x_var(ha_result["x"]), teams, distances), "\n")
#pprint.pprint(ha_result)

# Test 2: Non-HA-Opt
non_ha_result = solve_ttp_subproblem(
    mode = 'Non-HA-Opt',
    input_schedule = input_x_schedule,
    base_model_params = {
        'distances': formatted_distances,
        'n': n,
        'l': l,
        'u': u,
        'days': 2*(n - 1),
        'teams': range(1, n + 1),
        'days_x': range(1, 2*(n - 1) + 1),
        'days_y': range(1, 2*(n - 1)),
        'days_u': range(1, 2*(n - 1) - u + 1),
        'days_l': range(1, 2*(n - 1) - l + 1)
    }
)
print("Non-HA-Opt Solution:", get_sol_from_x_var(non_ha_result["x"]))
print("Total distance of Non-HA-Opt solution:", calculate_total_distance(get_sol_from_x_var(non_ha_result["x"]), teams, distances))
#pprint.pprint(non_ha_result)

Initial solution: {0: [(1, 4), (3, 2), (7, 0), (5, 6)], 1: [(5, 0), (1, 6), (3, 4), (7, 2)], 2: [(1, 3), (7, 5), (4, 0), (6, 2)], 3: [(6, 7), (2, 3), (0, 5), (4, 1)], 4: [(0, 1), (4, 5), (2, 6), (3, 7)], 5: [(1, 2), (5, 7), (3, 6), (0, 4)], 6: [(3, 1), (6, 0), (2, 5), (7, 4)], 7: [(0, 2), (6, 4), (7, 3), (5, 1)], 8: [(4, 2), (0, 6), (5, 3), (1, 7)], 9: [(3, 0), (4, 6), (2, 7), (1, 5)], 10: [(1, 0), (2, 4), (3, 5), (7, 6)], 11: [(5, 4), (2, 0), (6, 3), (7, 1)], 12: [(5, 2), (0, 3), (4, 7), (6, 1)], 13: [(2, 1), (4, 3), (0, 7), (6, 5)]}
Distance of initial solution: 43151
Formatted distances: {(5, 5): 0, (5, 3): 1020, (5, 2): 1090, (5, 7): 957, (5, 4): 1380, (5, 8): 1190, (5, 1): 605, (5, 6): 1010, (3, 5): 1020, (3, 3): 0, (3, 2): 80, (3, 7): 501, (3, 4): 380, (3, 8): 664, (3, 1): 665, (3, 6): 257, (2, 5): 1090, (2, 3): 80, (2, 2): 0, (2, 7): 567, (2, 4): 337, (2, 8): 712, (2, 1): 745, (2, 6): 315, (7, 5): 957, (7, 3): 501, (7, 2): 567, (7, 7): 0, (7, 4): 622, (7, 8): 250, (7, 1): 370, (